In [1]:
import os
os.chdir("/home/jakob/doktor/projects/EnsembleUncertainty/code")
"""Learing "logit" distribution in regression example"""
import logging
import zipfile
from copy import copy, deepcopy
import urllib.request
from pathlib import Path
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Optimizer
from torch.optim.sgd import SGD
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from sklearn.model_selection import KFold
from src.dataloaders.uci import uci_base, wine, bost
from src import metrics
from src import utils
from src.ensemble import simple_regressor, ensemble
from src import loss as custom_loss

# Settings
class Args():
    pass
args = Args()
args.seed = 1
args.gpu = False
args.log_dir = Path("./logs")
args.log_level = logging.INFO
args.retrain = True
args.num_ensemble_members=1
args.num_epochs=1
args.lr = 0.01




LOGGER = logging.getLogger(__name__)
EXPERIMENT_NAME = "red_regression_logits"

log_file = Path("{}_{}.log".format(
    EXPERIMENT_NAME,
    datetime.now().strftime("%Y%m%d_%H%M%S")))
utils.setup_logger(log_path=Path.cwd() / args.log_dir / log_file,
                   log_level=args.log_level)

# General constructs
train_metrics = list()
test_metrics = list()

rmse = metrics.Metric(name="RMSE", function=metrics.root_mean_squared_error)
train_metrics.append(deepcopy(rmse))
test_metrics.append(rmse)

BATCH_SIZE = 32
torch.cuda.device(0)
torch.cuda.get_device_name(torch.cuda.current_device())
device = torch.device("cuda")
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'svg'

2020-02-12 09:09:48,318 INFO  root            - Log at /home/jakob/doktor/projects/EnsembleUncertainty/code/logs/red_regression_logits_20200212_090948.log


In [2]:
def to_variable(var=(), cuda=True, volatile=False):
    out = []
    for v in var:
        
        if isinstance(v, np.ndarray):
            v = torch.from_numpy(v).type(torch.FloatTensor)

        if not v.is_cuda and cuda:
            v = v.cuda()

        if not isinstance(v, Variable):
            v = Variable(v, volatile=volatile)

        out.append(v)
    return out

def get_loss_and_rmse(network,
                      loss_function,
                      x, y,
                      mean_shift=None, std_scale=None):
    x, y = to_variable(var=(x, y), cuda=True)

    logits = network.forward(x)
    output = network.transform_logits(logits)
    mean, std = output

    if mean_shift is not None and std_scale is not None:
        mean_shift = torch.tensor(mean_shift).float().cuda()
        std_scale = torch.tensor(std_scale).float().cuda()
        mean = mean * std_scale + mean_shift
        y = y * std_scale + mean_shift
        std *= std_scale

    loss = loss_function((mean, std), y)

    rmse = ((mean - y)**2).mean()**0.5

    return loss.detach().cpu(), rmse.detach().cpu()

def create_ensemble(num_ensemble_members,
                    model,
                    ensemble_output_size):
    prob_ensemble = ensemble.Ensemble(ensemble_output_size)
    for _ in range(num_ensemble_members):
        prob_ensemble.add_member(copy(model))
    return prob_ensemble

def mean_and_std_from_list(samples):
    """Calculate mean and std from np-array compatible list"""
    array = np.array(samples)
    return array.mean(), array.std()

def mean_and_std_from_metric(metric, rescale=1):
    """Calculate mean and std from np-array compatible list"""
    return metric.mean() * rescale, metric.std() * rescale

# UCI datasets

In [3]:
def train_mc_dropout(data,
                     drop_prob,
                     num_epochs,
                     num_units,
                     learn_rate,
                     weight_decay,
                     train_metrics,
                     test_metrics,
                     batch_size):
    
    train_nll, test_nll = [], []
    train_rmses, test_rmses = [], []
    for metric in train_metrics:
        metric.reset()        
        
    for metric in test_metrics:
        metric.reset()

    network = simple_regressor.Model(layer_sizes=[data.input_dim, num_units, 2],
                                     device=device,
                                     variance_transform=utils.variance_linear_asymptote(),
                                     loss_function=custom_loss.gaussian_neg_log_likelihood_1d)
    network.optimizer = torch.optim.SGD(network.parameters(),
                                lr=learn_rate,
                                weight_decay=weight_decay)
    
    prob_ensemble = create_ensemble(num_ensemble_members=args.num_ensemble_members,
                               model=network,
                               ensemble_output_size=1)
    prob_ensemble.add_metrics(train_metrics)

    x_train, y_train, x_test, y_test = data.create_train_val_split(0.9)

    x_means, x_stds = x_train.mean(axis = 0), x_train.var(axis = 0)**0.5
    y_means, y_stds = y_train.mean(axis = 0), y_train.var(axis = 0)**0.5

    x_train = (x_train - x_means) / x_stds
    y_train = (y_train - y_means) / y_stds

    x_test = (x_test - x_means) / x_stds
    y_test = (y_test - y_means) / y_stds
    
    if batch_size is None:
        batch_size = x_train.shape[0]
    trainloader = uci_base.uci_dataloader(x_train, y_train, batch_size)

    losses = []
    prob_ensemble.train(train_loader=trainloader,
                        num_epochs=num_epochs)

    mean_shift = None
    std_scale = None
    #mean_shift = y_means
    #std_scale = y_stds

    train_loss, train_rmse = get_loss_and_rmse(network,
                                               network.loss,
                                               x_train,
                                               y_train,
                                               mean_shift=mean_shift,
                                               std_scale=std_scale)

    test_loss, test_rmse = get_loss_and_rmse(network,
                                             network.loss,
                                             x_test,
                                             y_test,
                                             mean_shift=mean_shift,
                                             std_scale=std_scale)

    testloader = uci_base.uci_dataloader(x_test, y_test, len(y_test))
    test_metrics, test_loss = network.test(testloader, test_metrics, network.loss)
    
    train_nll.append((train_loss.item()/len(x_train) + np.log(y_stds)[0]))
    test_nll.append((test_loss.item()/len(x_test) + np.log(y_stds)[0]))

    print("Train NLL\t = {:.3f} +/- {:.3f}".format(*mean_and_std_from_list(train_nll)))
    print("Test  NLL\t = {:.3f} +/- {:.3f}".format(*mean_and_std_from_list(test_nll)))
    print("Train RMSE\t = {:.3f} +/- {:.3f}".format(
        *mean_and_std_from_metric(train_metrics[0], rescale=y_stds[0])))
    print("Test RMSE\t = {:.3f} +/- {:.3f}".format(
        *mean_and_std_from_metric(test_metrics[0], rescale=y_stds[0])))
    return network

# Red wine dataset

In [6]:
wine_data = wine.WineData("data/uci/wine/winequality-red.csv")
net = train_mc_dropout(data=wine_data,
                       drop_prob=0.0,
                       num_epochs=40,
                       num_units=50,
                       learn_rate=1e-4,
                       weight_decay=0.0, #1e-1/len(data)**0.5,
                       train_metrics=train_metrics,
                       test_metrics=test_metrics,
                       batch_size=None)

2020-02-12 09:10:12,075 INFO  Model           - Moving model to device: cuda
2020-02-12 09:10:12,076 INFO  Model           - Using variance transform: <lambda>
2020-02-12 09:10:12,077 WARNING Ensemble        - Is subclass check disabled
2020-02-12 09:10:12,077 INFO  Ensemble        - Adding <class 'src.ensemble.simple_regressor.Model'> to ensemble
2020-02-12 09:10:12,078 INFO  Ensemble        - Adding metric: RMSE
2020-02-12 09:10:12,079 INFO  Ensemble        - Training ensemble
2020-02-12 09:10:12,079 INFO  Ensemble        - Training member 1/1
2020-02-12 09:10:12,093 INFO  Model           - Train - Epoch 1: Loss: 2128.920 RMSE: 0.991
2020-02-12 09:10:12,113 INFO  Model           - Train - Epoch 2: Loss: 1966.156 RMSE: 0.934
2020-02-12 09:10:12,127 INFO  Model           - Train - Epoch 3: Loss: 1898.120 RMSE: 0.902
2020-02-12 09:10:12,141 INFO  Model           - Train - Epoch 4: Loss: 1854.375 RMSE: 0.878
2020-02-12 09:10:12,155 INFO  Model           - Train - Epoch 5: Loss: 1822.426 

In [ ]:
net

# Housing dataset

In [ ]:
bost_data = bost.BostonData("data/uci/bost/housing.data")
train_mc_dropout(data=bost_data,
                 drop_prob=0.0,
                 num_epochs=40,
                 num_units=50,
                 learn_rate=1e-4,
                 weight_decay=0.0, #1e-1/len(data)**0.5,
                 train_metrics=train_metrics,
                 test_metrics=test_metrics,
                 batch_size=None)